Snake Game

### Primeiro, importamos as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
from glfw import *
from time import sleep

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(900, 900, "Cobrinha", None, None)
glfw.make_context_current(window)

#### Inicializando os shaders de vertices e de formação das primitivas geométricas (objetos na tela)

In [3]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader


In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader


In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [11]:
from enum import Enum

class GameObjectType(Enum):
    SNAKE, FOOD = list(range(2))

UP = [0, 1]
DOWN = [0, -1]
RIGHT = [1, 0]
LEFT = [-1, 0]

STD_SNAKE_SPEED = 0.0005
SQUARE_SCALE = 0.1
GRID_SIZE = int(2 / SQUARE_SCALE)
FRAMERATE = 4

## Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Aqui, temos as classes desenhadas para gerir o funcionamento do jogo.

In [12]:
def generate_square(coord_x: int, coord_y: int):
    square = np.zeros(4, [("position", np.float32, 2)])
    square['position'] = [
        (coord_x, coord_y),
        (coord_x, coord_y + SQUARE_SCALE),
        (coord_x + SQUARE_SCALE, coord_y),
        (coord_x + SQUARE_SCALE, coord_y + SQUARE_SCALE)
    ]
    return square

In [13]:
class GameObject:
    def __init__(self, obj_type: GameObjectType, coordinates: list[int, int]):
        self.type = obj_type
        self.square_vertices_matrix = generate_square(coordinates[0], coordinates[1])
        self.coordinates = coordinates
    
    def get_type(self) -> GameObjectType:
        return self.type
    
class Snake:
    def __init__(self, coordinates: list[int, int]):
        self.body: list[GameObject] = [GameObject(GameObjectType.SNAKE, coordinates)]
        self.body_len = 1

        self.snake_direction: tuple[int, int] = UP
        self.head_coordinates = coordinates
        
    def update_direction(self, new_direction: tuple[int, int]):
        self.snake_direction = new_direction
    
    def eat(self, food_position: list[int, int]) -> bool:
        if self.head_coordinates[0] == food_position[0] and self.head_coordinates[1] == food_position[1]:
            self.body_len += 1
            return True
        
        return False
    
    def hit(self) -> bool:
        for i in range(1, len(self.body)):
            if self.head_coordinates[0] == self.body[i].coordinates[0] and \
                self.head_coordinates[1] == self.body[i].coordinates[1]:
                
                return True
        
        return False

    def update_position(self):
        # If no food has been eaten.
        if self.body_len == len(self.body):
            for i in range(len(self.body) - 1):
                self.body[i].coordinates = self.body[i + 1].coordinates
        
            self.body[self.body_len - 1] = GameObject(GameObjectType.SNAKE, self.head_coordinates)
        
        else:
            self.body.append(GameObject(GameObjectType.SNAKE, self.head_coordinates))

        self.head_coordinates = [round(self.head_coordinates[i] + self.snake_direction[i] * SQUARE_SCALE, 2) for i in range(2)]

def new_food_coordinates():
    rand_x = int(np.random.random() * 20)
    rand_y = int(np.random.random() * 20)

    return [round(rand_x / 10 - 1, 2), round(rand_y / 10 - 1, 2)]

In [14]:
snake = Snake([0, 0])
food = GameObject(GameObjectType.FOOD, (new_food_coordinates()))

vertices = np.concatenate((snake.body[0].square_vertices_matrix, food.square_vertices_matrix))['position']

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [15]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [16]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [17]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Definimos essa variável no Vertex Shader.

In [18]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [19]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [20]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, exibimos a janela.


In [21]:
glfw.show_window(window)

- Aqui, temos a função que vai capturar e informar ao código quando um input for recebido do teclado.

In [22]:
def key_event(window,key,scancode,action,mods):
    if (action==1 or action==2):
        if key == 87 and snake.snake_direction != DOWN: # tecla W
            snake.update_direction(UP)    

        elif key == 83 and snake.snake_direction != UP: # tecla S
            snake.update_direction(DOWN)    
            
        elif key == 65 and snake.snake_direction != RIGHT: # tecla A
            snake.update_direction(LEFT)    

        elif key == 68 and snake.snake_direction != LEFT: # tecla D
            snake.update_direction(RIGHT)    
    

glfw.set_key_callback(window,key_event)

### Aplicação das matrizes de transformação

- Primeiro, define-se a matriz de translação baseado nas coordenadas passadas.

- Em seguida, essa matriz será usada na função print_object() para imprimir os objetos na tela conforme eles se movimentam.

In [23]:
def mat_translate(x, y):
    return np.array([[1.0, 0.0, 0.0, x],
                     [0.0, 1.0, 0.0, y],
                     [0.0, 0.0, 1.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0],
                     ], np.float32)

In [24]:
def print_object(coordinates: list, obj_type: GameObjectType):
    global mat_transform_snake, mat_transform_food

    loc = glGetUniformLocation(program, "mat_transformation")

    if obj_type == GameObjectType.SNAKE:
        mat_transform_snake = mat_translate(coordinates[0], coordinates[1])
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform_snake)   
        glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)

    elif obj_type == GameObjectType.FOOD:
        mat_transform_food = mat_translate(coordinates[0], coordinates[1])
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform_food)   
        glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)

mat_transform_snake = mat_translate(snake.head_coordinates[0], snake.head_coordinates[1])
mat_transform_food = mat_translate(food.coordinates[0], food.coordinates[1])

### Loop de execução

- O loop executará até que uma de tres condições sejam cumrpidas:
    - A janela do jogo é fechada;
    - A cobra acerta a si mesma;
    - A cobra acerta a 'parede' da janela do jogo.

- A primeira ação realizada a cada iteração é checar se a cobra comeu a maçã; caso isso aconteça, a cobra aumenta de tamanho e a maçã é aleatoriamente reposicionada.

- Em seguida, desenha-se na tela todas as partes da cobrinha.

- Por fim, desenha-se a maçã.

- Assim, após um tempo de espera (para evitar que o jogo fique acelerado), a próxima iteração do loop é chamada.

In [25]:
while not glfw.window_should_close(window) and not snake.hit() \
    and -1.0 <= snake.head_coordinates[0] <= 0.9 and -1.0 <= snake.head_coordinates[1] <= 0.9:
    
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    if snake.eat(food.coordinates):
        food.coordinates = new_food_coordinates()

    snake.update_position() 

    color_change = 0
    color_increment = 0
    for snake_part in snake.body:
        glUniform4f(loc_color, 0, 1 - color_increment, 0, 0.5)
        print_object(snake_part.coordinates, GameObjectType.SNAKE)
        if color_change == 0:
            color_increment += 0.05
            if color_increment >= 0.5:
                color_change = 1
        else:
            color_increment -= 0.05
            if color_increment == 0.0:
                color_change = 0
        
    
    glUniform4f(loc_color, 1, 0, 0, 0.5)
    print_object(food.coordinates, GameObjectType.FOOD)

    glfw.swap_buffers(window)

    sleep(1/FRAMERATE)

glfw.terminate()